# Import necessary libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

''''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename));'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"'import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename));"

In [2]:
# Load the data
df = pd.read_excel('/kaggle/input/twitter-dataset-for-sentiment-analysis/LabeledText.xlsx')

In [3]:
df

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,negative
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.txt,This is my Valentine's from 1 of my nephews. I...,positive
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,neutral
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,positive
...,...,...,...
4864,995.txt,OMG. Well done #Eskom! 'Man dies during #LoadS...,positive
4865,996.txt,Feelin' the love in here! #ValentinesDay #caring,positive
4866,997.txt,#blue #eyes can't be #beaten,neutral
4867,998.txt,LA CHUCHA LOUUU TE CHUPO LOS OJOS..!,neutral


# Preprocess the text data


In [4]:
# Preprocess the text data
tokenizer = Tokenizer(num_words=5000, split=' ')
tokenizer.fit_on_texts(df['Caption'].values)
X = tokenizer.texts_to_sequences(df['Caption'].values)
X = pad_sequences(X)

In [5]:
# Encode the labels
y = pd.get_dummies(df['LABEL']).values

# Split the data into training and testing sets


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the deep learning model architecture


In [7]:
# Define the deep learning model architecture
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Train the model on the training data


In [8]:
# Train the model on the training data
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
110/110 [==============================] - 50s 378ms/step - loss: 1.0718 - accuracy: 0.4126 - val_loss: 1.0475 - val_accuracy: 0.4615
Epoch 2/10
110/110 [==============================] - 37s 333ms/step - loss: 0.9504 - accuracy: 0.5281 - val_loss: 0.9985 - val_accuracy: 0.5385
Epoch 3/10
110/110 [==============================] - 38s 345ms/step - loss: 0.7084 - accuracy: 0.6845 - val_loss: 0.7721 - val_accuracy: 0.6795
Epoch 4/10
110/110 [==============================] - 34s 307ms/step - loss: 0.5183 - accuracy: 0.7957 - val_loss: 1.0286 - val_accuracy: 0.5538
Epoch 5/10
110/110 [==============================] - 34s 306ms/step - loss: 0.4019 - accuracy: 0.8514 - val_loss: 0.8379 - val_accuracy: 0.6718
Epoch 6/10
110/110 [==============================] - 33s 300ms/step - loss: 0.3215 - accuracy: 0.8842 - val_loss: 0.9265 - val_accuracy: 0.6872
Epoch 7/10
110/110 [==============================] - 32s 294ms/step - loss: 0.2514 - accuracy: 0.9136 - val_loss: 0.8813 - val_ac

# Evaluate the model on the testing data


In [9]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 1.1114857196807861
Test accuracy: 0.6919918060302734


# Save the trained model


In [10]:
# Save the trained model
model.save('sentiment_model.h5')

# Load the pre-trained VGG16 model


In [11]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Load an example image
image = load_img('/kaggle/input/twitter-dataset-for-sentiment-analysis/Images/Images/Negative/1008.jpg', target_size=(224, 224))

# Preprocess the image
image = img_to_array(image)
image = preprocess_input(image)
image = np.expand_dims(image, axis=0)

# Predict the class of the image
predictions = model.predict(image)
decoded_predictions = decode_predictions(predictions, top=3)[0]
print('Predictions:', decoded_predictions)


35363/35363 [==============================] - 0s 0us/step
Predictions: [('n03291819', 'envelope', 0.25218305), ('n07248320', 'book_jacket', 0.088189535), ('n02971356', 'carton', 0.064956166)]
